# 04 - Order Selector (Dispatch Generator)

## Overview

This notebook demonstrates the dispatch candidate generation system. We solve a knapsack-style optimization problem where we:

- **Maximize** total priority of selected orders
- **Respect** truck capacity constraints (7.0-8.5 pallets preferred, 9.0 hard max)
- **Include** all mandatory orders
- **Consider** zone coherence for routing efficiency

## Selection Strategies

| Strategy | Description |
|----------|-------------|
| Greedy Efficiency | Sort by priority/pallets ratio |
| Greedy Priority | Sort by priority only |
| Greedy Zone | Single zone selection (CABA, North, South, West) |
| Zone Spillover | Start with dominant zone, controlled expansion |
| Best Fit | Optimize for 8.0 pallet utilization |
| DP Optimal | Dynamic programming exact solution |
| Mandatory First | Start with mandatory, fill same zone |
| Mandatory Nearest | Geographic clustering around mandatory |

In [ ]:
# Setup and Imports
import sys
from pathlib import Path
import importlib

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Standard imports
import json
from datetime import datetime

# Data handling
import pandas as pd
import numpy as np

# Visualization (Plotly only - per project conventions)
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Reload module to get latest changes
import src.order_selector
importlib.reload(src.order_selector)

# Project modules
from src.database import get_database_manager
from src.order_selector import (
    SelectionStrategy,
    OrderForSelection,
    DispatchCandidate,
    SelectorConfig,
    load_selector_config,
    load_pending_orders,
    get_mandatory_orders,
    get_non_mandatory_orders,
    calculate_mandatory_pallets,
    select_mandatory_subset,
    get_mandatory_overflow_info,
    get_zone_breakdown,
    get_dominant_zone,
    greedy_by_efficiency,
    greedy_by_priority,
    greedy_by_zone,
    greedy_zone_with_spillover,
    greedy_best_fit,
    dp_optimal_knapsack,
    mandatory_first,
    greedy_mandatory_nearest,
    build_dispatch_candidate,
    generate_all_candidates,
    generate_all_candidates_with_info,
    generate_candidates_no_mandatory,
    deduplicate_candidates,
    rank_candidates,
    get_best_single_zone_candidate,
    get_exceptional_multizone_candidates,
    get_top_n_candidates,
    export_candidates_to_json,
    get_candidates_summary_df,
)

print("Imports loaded successfully!")

Imports loaded successfully!


In [2]:
# Path configuration
DATA_DIR = project_root / "data"
CONFIG_DIR = DATA_DIR / "config"
DB_PATH = DATA_DIR / "processed" / "delivery.db"
OUTPUT_DIR = project_root / "output"
DISPATCH_DIR = OUTPUT_DIR / "dispatches"

# Load configuration
CONFIG_PATH = CONFIG_DIR / "order_selector_config.json"
config = load_selector_config(CONFIG_PATH)

# Connect to database
db = get_database_manager(DB_PATH)

print(f"Database: {DB_PATH}")
print(f"Config: {CONFIG_PATH}")
print(f"\nCapacity Configuration:")
print(f"  Nominal: {config.nominal_capacity} pallets")
print(f"  Acceptable Range: {config.min_acceptable} - {config.max_acceptable} pallets")
print(f"  Hard Maximum: {config.hard_max} pallets")
print(f"  Min for Zone Candidate: {config.min_for_zone_candidate} pallets")

Database: c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\data\processed\delivery.db
Config: c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\data\config\order_selector_config.json

Capacity Configuration:
  Nominal: 8.0 pallets
  Acceptable Range: 7.0 - 8.5 pallets
  Hard Maximum: 9.0 pallets
  Min for Zone Candidate: 4.0 pallets


## 1. Load and Analyze Pending Orders

In [3]:
# Load pending orders
orders = load_pending_orders(db)

# Create DataFrame for analysis
orders_df = pd.DataFrame([
    {
        "order_id": o.order_id,
        "client_name": o.client_name,
        "total_pallets": o.total_pallets,
        "priority_score": o.priority_score,
        "zone_id": o.zone_id,
        "is_mandatory": o.is_mandatory,
        "efficiency": o.priority_score / max(o.total_pallets, 0.1),
    }
    for o in orders
])

print(f"Total Pending Orders: {len(orders)}")
print(f"Total Pallets Available: {orders_df['total_pallets'].sum():.2f}")
print(f"Total Priority Available: {orders_df['priority_score'].sum():.2f}")
print(f"\nPallet Statistics:")
print(f"  Min: {orders_df['total_pallets'].min():.2f}")
print(f"  Max: {orders_df['total_pallets'].max():.2f}")
print(f"  Mean: {orders_df['total_pallets'].mean():.2f}")
print(f"  Median: {orders_df['total_pallets'].median():.2f}")

Total Pending Orders: 41
Total Pallets Available: 151.22
Total Priority Available: 5002267.90

Pallet Statistics:
  Min: 0.71
  Max: 8.00
  Mean: 3.69
  Median: 3.48


In [4]:
# Display orders table
display_df = orders_df.sort_values("priority_score", ascending=False).head(15)
display_df

,order_id,client_name,total_pallets,priority_score,zone_id,is_mandatory,efficiency
0,ORD-AF4525B0,Distribuidora Pampa,4.06,999999.00,WEST_ZONE,True,246305.172414
3,ORD-930E42E5,Supermercado Don Pedro,3.94,999999.00,CABA,True,253806.852792
7,ORD-AC59CDC9,Autoservicio La Plaza,5.07,999999.00,WEST_ZONE,True,197238.461538
20,ORD-2F51CAB4,Almacen El Buen Precio,4.15,999999.00,SOUTH_ZONE,True,240963.614458
10,ORD-9D6187C8,Supermercado Don Pedro,4.19,999999.00,CABA,True,238663.245823
5,ORD-89270B2E,Almacen Mi Tierra,7.87,103.38,CABA,False,13.135959
13,ORD-A5A4DFFE,Comercial Rivadavia,4.89,102.68,SOUTH_ZONE,False,20.997955
24,ORD-6E478AEA,Autoservicio El Trebol,3.88,99.84,WEST_ZONE,False,25.731959
16,ORD-5FF28C36,Autoservicio La Plaza,1.12,96.54,WEST_ZONE,False,86.196429
36,ORD-AEEA364E,Mayorista Don Juan,5.29,93.89,NORTH_ZONE,False,17.748582


## 2. Mandatory Orders Analysis

In [5]:
# Identify mandatory orders
all_mandatory = get_mandatory_orders(orders)
total_mandatory_pallets = calculate_mandatory_pallets(all_mandatory)

print(f"Total Mandatory Orders: {len(all_mandatory)}")
print(f"Total Mandatory Pallets: {total_mandatory_pallets:.2f}")
print(f"Truck Hard Max Capacity: {config.hard_max} pallets")

# Handle mandatory overflow - select subset that fits
if total_mandatory_pallets > config.hard_max:
    print(f"\n⚠️ MANDATORY OVERFLOW: {total_mandatory_pallets:.2f} pallets > {config.hard_max} capacity!")
    print("   Will select subset of mandatory orders that fit in one dispatch.")
    
    mandatory, deferred_mandatory = select_mandatory_subset(
        all_mandatory, config.hard_max, strategy="priority"
    )
    mandatory_pallets = calculate_mandatory_pallets(mandatory)
    
    print(f"\n   ✓ Selected for this dispatch: {len(mandatory)} orders ({mandatory_pallets:.2f} pallets)")
    print(f"   ⏳ Deferred to next dispatch: {len(deferred_mandatory)} orders ({total_mandatory_pallets - mandatory_pallets:.2f} pallets)")
    
    if deferred_mandatory:
        print("\n   Deferred Mandatory Orders:")
        deferred_df = pd.DataFrame([
            {"order_id": o.order_id, "client": o.client_name[:25], "pallets": o.total_pallets, "zone": o.zone_id}
            for o in deferred_mandatory
        ])
        display(deferred_df)
else:
    mandatory = all_mandatory
    deferred_mandatory = []
    mandatory_pallets = total_mandatory_pallets
    print(f"\n✓ All mandatory orders fit within capacity ({mandatory_pallets:.2f} / {config.hard_max} pallets)")

print(f"\nRemaining Capacity after mandatory: {config.max_acceptable - mandatory_pallets:.2f} pallets")

# Mandatory orders breakdown (selected for this dispatch)
if mandatory:
    mandatory_df = pd.DataFrame([
        {
            "order_id": o.order_id,
            "client_name": o.client_name,
            "pallets": o.total_pallets,
            "priority_score": o.priority_score,
            "zone_id": o.zone_id,
        }
        for o in mandatory
    ])
    print("\nMandatory Orders for This Dispatch:")
    display(mandatory_df)
    
    # Zone breakdown
    mandatory_zones = get_zone_breakdown(mandatory)
    print(f"\nMandatory Zone Breakdown: {mandatory_zones}")
else:
    print("\nNo mandatory orders in pending queue.")

Total Mandatory Orders: 5
Total Mandatory Pallets: 21.41
Truck Hard Max Capacity: 9.0 pallets

⚠️ MANDATORY OVERFLOW: 21.41 pallets > 9.0 capacity!
   Will select subset of mandatory orders that fit in one dispatch.

   ✓ Selected for this dispatch: 2 orders (8.00 pallets)
   ⏳ Deferred to next dispatch: 3 orders (13.41 pallets)

   Deferred Mandatory Orders:


,order_id,client,pallets,zone
0,ORD-AC59CDC9,Autoservicio La Plaza,5.07,WEST_ZONE
1,ORD-9D6187C8,Supermercado Don Pedro,4.19,CABA
2,ORD-2F51CAB4,Almacen El Buen Precio,4.15,SOUTH_ZONE



Remaining Capacity after mandatory: 0.50 pallets

Mandatory Orders for This Dispatch:


,order_id,client_name,pallets,priority_score,zone_id
0,ORD-AF4525B0,Distribuidora Pampa,4.06,999999.0,WEST_ZONE
1,ORD-930E42E5,Supermercado Don Pedro,3.94,999999.0,CABA



Mandatory Zone Breakdown: {'WEST_ZONE': 1, 'CABA': 1}


## 3. Zone Distribution Overview

In [6]:
# Aggregate by zone
zone_summary = orders_df.groupby("zone_id").agg(
    order_count=("order_id", "count"),
    total_pallets=("total_pallets", "sum"),
    total_priority=("priority_score", "sum"),
    avg_efficiency=("efficiency", "mean"),
).reset_index()

zone_summary["priority_per_pallet"] = zone_summary["total_priority"] / zone_summary["total_pallets"]
zone_summary = zone_summary.round(2)

print("Zone Summary:")
display(zone_summary)

Zone Summary:


,zone_id,order_count,total_pallets,total_priority,avg_efficiency,priority_per_pallet
0,CABA,11,45.75,2000529.29,44783.65,43727.42
1,NORTH_ZONE,5,14.02,295.83,26.57,21.10
2,SOUTH_ZONE,7,23.75,1000365.02,34439.20,42120.63
3,WEST_ZONE,18,67.70,2001077.76,24666.53,29558.02


In [7]:
# Zone color mapping
zone_colors = {
    "CABA": "#FF6B6B",
    "NORTH_ZONE": "#4ECDC4",
    "SOUTH_ZONE": "#45B7D1",
    "WEST_ZONE": "#96CEB4",
}

# Create subplot with 3 bar charts
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Orders per Zone", "Pallets per Zone", "Priority per Zone"),
    horizontal_spacing=0.08,
)

# Orders count
fig.add_trace(
    go.Bar(
        x=zone_summary["zone_id"],
        y=zone_summary["order_count"],
        marker_color=[zone_colors.get(z, "#888") for z in zone_summary["zone_id"]],
        text=zone_summary["order_count"],
        textposition="outside",
        name="Orders",
    ),
    row=1, col=1,
)

# Pallets
fig.add_trace(
    go.Bar(
        x=zone_summary["zone_id"],
        y=zone_summary["total_pallets"],
        marker_color=[zone_colors.get(z, "#888") for z in zone_summary["zone_id"]],
        text=zone_summary["total_pallets"].round(1),
        textposition="outside",
        name="Pallets",
    ),
    row=1, col=2,
)

# Priority
fig.add_trace(
    go.Bar(
        x=zone_summary["zone_id"],
        y=zone_summary["total_priority"],
        marker_color=[zone_colors.get(z, "#888") for z in zone_summary["zone_id"]],
        text=zone_summary["total_priority"].round(0),
        textposition="outside",
        name="Priority",
    ),
    row=1, col=3,
)

fig.update_layout(
    title="Zone Distribution Analysis",
    showlegend=False,
    height=400,
)

fig.show()

In [8]:
# Identify zone with most potential
dominant_zone = get_dominant_zone(orders)
print(f"Dominant Zone (highest total priority): {dominant_zone}")

# Zone efficiency comparison
fig = px.bar(
    zone_summary,
    x="zone_id",
    y="priority_per_pallet",
    color="zone_id",
    color_discrete_map=zone_colors,
    title="Priority Efficiency by Zone (Priority per Pallet)",
    text="priority_per_pallet",
)
fig.update_traces(texttemplate="%{text:.1f}", textposition="outside")
fig.update_layout(showlegend=False, height=400)
fig.show()

Dominant Zone (highest total priority): WEST_ZONE


## 4. Single Strategy Demo: Zone Spillover

Let's walk through the **Greedy Zone with Spillover** strategy step-by-step to understand how it makes decisions.

In [9]:
# Step 1: Identify dominant zone
print("=" * 60)
print("GREEDY ZONE SPILLOVER - Step by Step")
print("=" * 60)

# Calculate zone priorities
zone_priorities = {}
for order in orders:
    zone_priorities[order.zone_id] = zone_priorities.get(order.zone_id, 0) + order.priority_score

print("\n📊 Step 1: Zone Priority Analysis")
for zone, priority in sorted(zone_priorities.items(), key=lambda x: -x[1]):
    print(f"   {zone}: {priority:.2f} total priority")

dominant = max(zone_priorities, key=zone_priorities.get)
print(f"\n✓ Dominant Zone: {dominant}")

GREEDY ZONE SPILLOVER - Step by Step

📊 Step 1: Zone Priority Analysis
   WEST_ZONE: 2001077.76 total priority
   CABA: 2000529.29 total priority
   SOUTH_ZONE: 1000365.02 total priority
   NORTH_ZONE: 295.83 total priority

✓ Dominant Zone: WEST_ZONE


In [10]:
# Step 2: Fill from dominant zone
print("\n📦 Step 2: Fill from Dominant Zone")

mandatory_ids = {o.order_id for o in mandatory}
dominant_orders = [
    o for o in orders 
    if o.zone_id == dominant and o.order_id not in mandatory_ids
]
dominant_orders.sort(key=lambda o: o.priority_score, reverse=True)

selected = list(mandatory)
current_pallets = mandatory_pallets

print(f"   Starting with {len(mandatory)} mandatory orders ({mandatory_pallets:.2f} pallets)")
print(f"   Available in {dominant}: {len(dominant_orders)} orders")

for order in dominant_orders:
    if current_pallets + order.total_pallets <= config.max_acceptable:
        selected.append(order)
        current_pallets += order.total_pallets
        print(f"   ✓ Added: {order.order_id} ({order.total_pallets:.2f}p, {order.priority_score:.1f} pri)")
    elif current_pallets >= config.min_acceptable:
        print(f"   ⊘ Capacity reached ({current_pallets:.2f} pallets)")
        break

print(f"\n   After dominant zone: {len(selected)} orders, {current_pallets:.2f} pallets")


📦 Step 2: Fill from Dominant Zone
   Starting with 2 mandatory orders (8.00 pallets)
   Available in WEST_ZONE: 17 orders
   ⊘ Capacity reached (8.00 pallets)

   After dominant zone: 2 orders, 8.00 pallets


In [11]:
# Step 3: Check spillover conditions
print("\n🔄 Step 3: Spillover Decision")

remaining_capacity = config.max_acceptable - current_pallets
print(f"   Remaining capacity: {remaining_capacity:.2f} pallets")
print(f"   Spillover threshold: {config.spillover_capacity_threshold} pallets")

if remaining_capacity > config.spillover_capacity_threshold:
    print(f"   ✓ Spillover TRIGGERED (remaining > threshold)")
    
    # Get adjacent zones
    adjacent = config.zone_adjacency.get(dominant, [])
    print(f"   Adjacent zones: {adjacent}")
    
    current_priority = sum(o.priority_score for o in selected)
    selected_ids = {o.order_id for o in selected}
    
    adjacent_orders = [
        o for o in orders
        if o.zone_id in adjacent and o.order_id not in selected_ids
    ]
    adjacent_orders.sort(key=lambda o: o.priority_score, reverse=True)
    
    print(f"\n   Evaluating {len(adjacent_orders)} adjacent zone orders:")
    
    for order in adjacent_orders[:5]:  # Show first 5
        if current_pallets + order.total_pallets <= config.max_acceptable:
            marginal_increase = order.priority_score / max(current_priority, 1)
            if marginal_increase >= config.spillover_priority_threshold:
                print(f"   ✓ {order.order_id} [{order.zone_id}]: +{marginal_increase:.1%} priority (meets {config.spillover_priority_threshold:.0%} threshold)")
                selected.append(order)
                current_pallets += order.total_pallets
                current_priority += order.priority_score
            else:
                print(f"   ✗ {order.order_id} [{order.zone_id}]: +{marginal_increase:.1%} priority (below threshold)")
else:
    print(f"   ✗ Spillover NOT triggered (remaining < threshold)")


🔄 Step 3: Spillover Decision
   Remaining capacity: 0.50 pallets
   Spillover threshold: 2.0 pallets
   ✗ Spillover NOT triggered (remaining < threshold)


In [12]:
# Build candidate from step-by-step result
demo_candidate = build_dispatch_candidate(selected, SelectionStrategy.GREEDY_ZONE_SPILLOVER, config)

print("\n" + "=" * 60)
print("FINAL RESULT")
print("=" * 60)
if demo_candidate:
    print(f"Candidate ID: {demo_candidate.candidate_id}")
    print(f"Orders: {len(demo_candidate.order_ids)}")
    print(f"Pallets: {demo_candidate.total_pallets} ({demo_candidate.utilization_pct}% utilization)")
    print(f"Priority: {demo_candidate.total_priority} (adjusted: {demo_candidate.adjusted_priority})")
    print(f"Zones: {demo_candidate.zones}")
    print(f"Zone Breakdown: {demo_candidate.zone_breakdown}")
    print(f"Single Zone: {demo_candidate.is_single_zone}")
    print(f"Zone Penalty: {demo_candidate.zone_dispersion_penalty}")


FINAL RESULT
Candidate ID: DISP-20260119-GREEDY-E784
Orders: 2
Pallets: 8.0 (100.0% utilization)
Priority: 1999998.0 (adjusted: 1899998.1)
Zones: ['WEST_ZONE', 'CABA']
Zone Breakdown: {'WEST_ZONE': 1, 'CABA': 1}
Single Zone: False
Zone Penalty: 0.95


## 5. Generate All Candidates

In [13]:
# Generate candidates using all strategies
# The function now handles mandatory overflow by selecting a subset that fits
all_candidates, generation_info = generate_all_candidates_with_info(db, CONFIG_PATH)

# Check if there was mandatory overflow
overflow_info = generation_info["overflow_info"]
deferred_mandatory = generation_info["deferred_mandatory"]

if overflow_info["requires_multiple_dispatches"]:
    print("⚠️ MANDATORY ORDER OVERFLOW DETECTED")
    print(f"   Total mandatory: {overflow_info['total_mandatory_orders']} orders ({overflow_info['total_mandatory_pallets']} pallets)")
    print(f"   Truck capacity: {overflow_info['max_capacity']} pallets")
    print(f"   Overflow: {overflow_info['overflow_pallets']} pallets")
    print(f"   Estimated dispatches needed: {overflow_info['estimated_dispatches_needed']}")
    print(f"\n   ✓ Included in this dispatch: {generation_info['included_mandatory_count']} mandatory orders")
    print(f"   ⏳ Deferred to next dispatch: {generation_info['deferred_mandatory_count']} mandatory orders")
    
    if deferred_mandatory:
        print("\n   Deferred Mandatory Orders:")
        for o in deferred_mandatory:
            print(f"      - {o.order_id}: {o.client_name[:30]} | {o.total_pallets:.2f}p | {o.zone_id}")
else:
    print("✓ All mandatory orders fit within capacity")

print(f"\nGenerated {len(all_candidates)} candidates from all strategies:")
for c in all_candidates:
    print(f"  - {c.strategy.value}: {len(c.order_ids)} orders, {c.total_pallets}p, {c.total_priority:.0f} priority")

⚠️ MANDATORY ORDER OVERFLOW DETECTED
   Total mandatory: 5 orders (21.41 pallets)
   Truck capacity: 9.0 pallets
   Overflow: 12.41 pallets
   Estimated dispatches needed: 3

   ✓ Included in this dispatch: 2 mandatory orders
   ⏳ Deferred to next dispatch: 3 mandatory orders

   Deferred Mandatory Orders:
      - ORD-AC59CDC9: Autoservicio La Plaza | 5.07p | WEST_ZONE
      - ORD-9D6187C8: Supermercado Don Pedro | 4.19p | CABA
      - ORD-2F51CAB4: Almacen El Buen Precio | 4.15p | SOUTH_ZONE

Generated 11 candidates from all strategies:
  - greedy_efficiency: 2 orders, 8.0p, 1999998 priority
  - greedy_priority: 2 orders, 8.0p, 1999998 priority
  - greedy_zone_caba: 3 orders, 8.78p, 1000126 priority
  - greedy_zone_north: 3 orders, 8.77p, 209 priority
  - greedy_zone_south: 2 orders, 8.18p, 175 priority
  - greedy_zone_west: 2 orders, 7.94p, 1000099 priority
  - greedy_zone_spillover: 2 orders, 8.0p, 1999998 priority
  - greedy_best_fit: 2 orders, 8.0p, 1999998 priority
  - dp_optimal

In [14]:
# Summary DataFrame before deduplication
raw_summary_df = get_candidates_summary_df(all_candidates)
print("\nAll Candidates (before deduplication):")
display(raw_summary_df)


All Candidates (before deduplication):


,candidate_id,strategy,total_pallets,adjusted_priority,total_priority,utilization_pct,zones,zone_count,is_single_zone,order_count,mandatory_count
8,DISP-20260119-DP_OPT-1710,dp_optimal,8.71,1900070.74,2000074.46,108.9,"WEST_ZONE, CABA",2,False,3,2
1,DISP-20260119-GREEDY-CA02,greedy_priority,8.00,1899998.10,1999998.00,100.0,"WEST_ZONE, CABA",2,False,2,2
0,DISP-20260119-GREEDY-D1C6,greedy_efficiency,8.00,1899998.10,1999998.00,100.0,"WEST_ZONE, CABA",2,False,2,2
10,DISP-20260119-GREEDY-5D25,greedy_mandatory_nearest,8.00,1899998.10,1999998.00,100.0,"WEST_ZONE, CABA",2,False,2,2
6,DISP-20260119-GREEDY-0275,greedy_zone_spillover,8.00,1899998.10,1999998.00,100.0,"WEST_ZONE, CABA",2,False,2,2
7,DISP-20260119-GREEDY-59A6,greedy_best_fit,8.00,1899998.10,1999998.00,100.0,"WEST_ZONE, CABA",2,False,2,2
9,DISP-20260119-MANDAT-9101,mandatory_first,8.00,1899998.10,1999998.00,100.0,"WEST_ZONE, CABA",2,False,2,2
2,DISP-20260119-GREEDY-30D3,greedy_zone_caba,8.78,1000126.17,1000126.17,109.7,CABA,1,True,3,1
5,DISP-20260119-GREEDY-A730,greedy_zone_west,7.94,1000098.84,1000098.84,99.2,WEST_ZONE,1,True,2,1
3,DISP-20260119-GREEDY-C834,greedy_zone_north,8.77,208.75,208.75,109.6,NORTH_ZONE,1,True,3,0


## 6. Deduplication Analysis

In [15]:
# Deduplicate candidates
unique_candidates = deduplicate_candidates(all_candidates)

print(f"Before deduplication: {len(all_candidates)} candidates")
print(f"After deduplication: {len(unique_candidates)} unique candidates")
print(f"Duplicates removed: {len(all_candidates) - len(unique_candidates)}")

# Identify which strategies produced duplicates
order_sets = {}
for c in all_candidates:
    key = frozenset(c.order_ids)
    if key not in order_sets:
        order_sets[key] = []
    order_sets[key].append(c.strategy.value)

print("\nStrategies producing same results:")
for order_set, strategies in order_sets.items():
    if len(strategies) > 1:
        print(f"  - {strategies}")

Before deduplication: 11 candidates
After deduplication: 6 unique candidates
Duplicates removed: 5

Strategies producing same results:
  - ['greedy_efficiency', 'greedy_priority', 'greedy_zone_spillover', 'greedy_best_fit', 'mandatory_first', 'greedy_mandatory_nearest']


In [16]:
# Generate candidates WITHOUT mandatory orders for comparison charts
non_mandatory_candidates = generate_candidates_no_mandatory(db, CONFIG_PATH)
unique_non_mandatory = deduplicate_candidates(non_mandatory_candidates)
non_mandatory_summary_df = get_candidates_summary_df(unique_non_mandatory, include_order_ids=True)

print(f"Non-mandatory candidates: {len(non_mandatory_candidates)} total, {len(unique_non_mandatory)} unique")

Non-mandatory candidates: 9 total, 7 unique


## 7. Candidates Comparison

In [17]:
# Summary DataFrame with order IDs
summary_df = get_candidates_summary_df(unique_candidates, include_order_ids=True)
print("Unique Candidates Summary:")
display(summary_df)

Unique Candidates Summary:


,candidate_id,strategy,total_pallets,adjusted_priority,total_priority,utilization_pct,zones,zone_count,is_single_zone,order_count,mandatory_count,order_ids
5,DISP-20260119-DP_OPT-1710,dp_optimal,8.71,1900070.74,2000074.46,108.9,"WEST_ZONE, CABA",2,False,3,2,"ORD-AF4525B0, ORD-930E42E5, ORD-63A5B648"
0,DISP-20260119-GREEDY-D1C6,greedy_efficiency,8.00,1899998.10,1999998.00,100.0,"WEST_ZONE, CABA",2,False,2,2,"ORD-AF4525B0, ORD-930E42E5"
1,DISP-20260119-GREEDY-30D3,greedy_zone_caba,8.78,1000126.17,1000126.17,109.7,CABA,1,True,3,1,"ORD-930E42E5, ORD-B24A7359, ORD-BCF7DBFD"
4,DISP-20260119-GREEDY-A730,greedy_zone_west,7.94,1000098.84,1000098.84,99.2,WEST_ZONE,1,True,2,1,"ORD-AF4525B0, ORD-6E478AEA"
2,DISP-20260119-GREEDY-C834,greedy_zone_north,8.77,208.75,208.75,109.6,NORTH_ZONE,1,True,3,0,"ORD-AEEA364E, ORD-FE3E2E80, ORD-8061B3FC"
3,DISP-20260119-GREEDY-B53E,greedy_zone_south,8.18,175.02,175.02,102.2,SOUTH_ZONE,1,True,2,0,"ORD-A5A4DFFE, ORD-A85586B6"


In [18]:
# Scatter: Priority vs Utilization
fig = px.scatter(
    summary_df,
    x="utilization_pct",
    y="adjusted_priority",
    color="zone_count",
    size="order_count",
    hover_data=["strategy", "zones", "total_pallets"],
    title="Candidate Comparison: Priority vs Utilization",
    labels={
        "utilization_pct": "Utilization (%)",
        "adjusted_priority": "Adjusted Priority",
        "zone_count": "Zone Count",
    },
    color_continuous_scale="RdYlGn_r",
)

# Add reference lines for capacity range
fig.add_vline(x=87.5, line_dash="dash", line_color="gray", annotation_text="Min (7.0p)")
fig.add_vline(x=106.25, line_dash="dash", line_color="gray", annotation_text="Max (8.5p)")

fig.update_layout(height=500)
fig.show()

In [19]:
# Bar chart: Adjusted priority per candidate
fig = px.bar(
    summary_df.sort_values("adjusted_priority", ascending=False),
    x="strategy",
    y="adjusted_priority",
    color="is_single_zone",
    title="Adjusted Priority by Strategy",
    color_discrete_map={True: "#4ECDC4", False: "#FF6B6B"},
    labels={"is_single_zone": "Single Zone"},
)
fig.update_layout(height=450, xaxis_tickangle=45)
fig.show()

In [20]:
# Zone composition per candidate (stacked bar)
zone_data = []
for c in unique_candidates:
    for zone, count in c.zone_breakdown.items():
        zone_data.append({
            "strategy": c.strategy.value,
            "zone": zone,
            "order_count": count,
        })

zone_df = pd.DataFrame(zone_data)

fig = px.bar(
    zone_df,
    x="strategy",
    y="order_count",
    color="zone",
    title="Zone Composition by Strategy",
    color_discrete_map=zone_colors,
)
fig.update_layout(height=450, xaxis_tickangle=45, barmode="stack")
fig.show()

### Comparison Charts (Without Mandatory Orders)

The following charts show the same visualizations but using candidates generated **without mandatory orders**. This allows us to see the real priority distribution without the 999999 priority boost from mandatory flags.

In [21]:
# Scatter: Priority vs Utilization (Non-Mandatory)
fig = px.scatter(
    non_mandatory_summary_df,
    x="utilization_pct",
    y="adjusted_priority",
    color="zone_count",
    size="order_count",
    hover_data=["strategy", "zones", "total_pallets"],
    title="Candidate Comparison: Priority vs Utilization (WITHOUT Mandatory)",
    labels={
        "utilization_pct": "Utilization (%)",
        "adjusted_priority": "Adjusted Priority (Real Scores)",
        "zone_count": "Zone Count",
    },
    color_continuous_scale="RdYlGn_r",
)

# Add reference lines for capacity range
fig.add_vline(x=87.5, line_dash="dash", line_color="gray", annotation_text="Min (7.0p)")
fig.add_vline(x=106.25, line_dash="dash", line_color="gray", annotation_text="Max (8.5p)")

fig.update_layout(height=500)
fig.show()

In [22]:
# Bar chart: Adjusted priority per candidate (Non-Mandatory)
fig = px.bar(
    non_mandatory_summary_df.sort_values("adjusted_priority", ascending=False),
    x="strategy",
    y="adjusted_priority",
    color="is_single_zone",
    title="Adjusted Priority by Strategy (WITHOUT Mandatory)",
    color_discrete_map={True: "#4ECDC4", False: "#FF6B6B"},
    labels={"is_single_zone": "Single Zone"},
)
fig.update_layout(height=450, xaxis_tickangle=45)
fig.show()

In [23]:
# Zone composition per candidate (Non-Mandatory)
zone_data_nm = []
for c in unique_non_mandatory:
    for zone, count in c.zone_breakdown.items():
        zone_data_nm.append({
            "strategy": c.strategy.value,
            "zone": zone,
            "order_count": count,
        })

zone_df_nm = pd.DataFrame(zone_data_nm)

fig = px.bar(
    zone_df_nm,
    x="strategy",
    y="order_count",
    color="zone",
    title="Zone Composition by Strategy (WITHOUT Mandatory)",
    color_discrete_map=zone_colors,
)
fig.update_layout(height=450, xaxis_tickangle=45, barmode="stack")
fig.show()

## 8. Zone Coherence Analysis

In [24]:
# Best single-zone candidate
best_single = get_best_single_zone_candidate(unique_candidates)

if best_single:
    print("🏆 Best Single-Zone Candidate:")
    print(f"   Strategy: {best_single.strategy.value}")
    print(f"   Zone: {best_single.zones[0]}")
    print(f"   Orders: {len(best_single.order_ids)}")
    print(f"   Pallets: {best_single.total_pallets} ({best_single.utilization_pct}%)")
    print(f"   Priority: {best_single.total_priority} (adjusted: {best_single.adjusted_priority})")
else:
    print("No single-zone candidates available.")

🏆 Best Single-Zone Candidate:
   Strategy: greedy_zone_caba
   Zone: CABA
   Orders: 3
   Pallets: 8.78 (109.7%)
   Priority: 1000126.17 (adjusted: 1000126.17)


In [25]:
# Exceptional multi-zone candidates
if best_single:
    exceptional = get_exceptional_multizone_candidates(
        unique_candidates,
        best_single.adjusted_priority,
        config.multizone_exception_threshold,
    )
    
    print(f"\n🌐 Exceptional Multi-Zone Candidates (>{config.multizone_exception_threshold:.0%} above best single):")
    if exceptional:
        for c in exceptional:
            improvement = (c.total_priority / best_single.adjusted_priority - 1) * 100
            print(f"   - {c.strategy.value}: {c.total_priority:.0f} priority (+{improvement:.1f}%)")
            print(f"     Zones: {c.zones}")
    else:
        print("   None - single-zone solutions are optimal for this order set.")


🌐 Exceptional Multi-Zone Candidates (>30% above best single):
   - greedy_efficiency: 1999998 priority (+100.0%)
     Zones: ['WEST_ZONE', 'CABA']
   - dp_optimal: 2000074 priority (+100.0%)
     Zones: ['WEST_ZONE', 'CABA']


In [26]:
# Compare best single vs best multi-zone
multi_zone_candidates = [c for c in unique_candidates if not c.is_single_zone]
best_multi = max(multi_zone_candidates, key=lambda c: c.adjusted_priority) if multi_zone_candidates else None

if best_single and best_multi:
    comparison_data = {
        "Metric": ["Strategy", "Orders", "Pallets", "Utilization", "Priority", "Adjusted Priority", "Zones"],
        "Best Single-Zone": [
            best_single.strategy.value,
            len(best_single.order_ids),
            best_single.total_pallets,
            f"{best_single.utilization_pct}%",
            best_single.total_priority,
            best_single.adjusted_priority,
            best_single.zones[0],
        ],
        "Best Multi-Zone": [
            best_multi.strategy.value,
            len(best_multi.order_ids),
            best_multi.total_pallets,
            f"{best_multi.utilization_pct}%",
            best_multi.total_priority,
            best_multi.adjusted_priority,
            ", ".join(best_multi.zones),
        ],
    }
    comparison_df = pd.DataFrame(comparison_data)
    print("\n📊 Single vs Multi-Zone Comparison:")
    display(comparison_df)


📊 Single vs Multi-Zone Comparison:


,Metric,Best Single-Zone,Best Multi-Zone
0,Strategy,greedy_zone_caba,dp_optimal
1,Orders,3,3
2,Pallets,8.78,8.71
3,Utilization,109.7%,108.9%
4,Priority,1000126.17,2000074.46
5,Adjusted Priority,1000126.17,1900070.74
6,Zones,CABA,"WEST_ZONE, CABA"


## 9. Ranking and Final Selection

In [27]:
# Rank candidates
ranked_candidates = rank_candidates(unique_candidates, config)

print("Candidates Ranked by Combined Score:")
print(f"  Weights: Priority={config.ranking_weight_priority}, "
      f"Utilization={config.ranking_weight_utilization}, "
      f"Zone Coherence={config.ranking_weight_zone_coherence}")
print()

for i, c in enumerate(ranked_candidates, 1):
    zone_label = "✓ Single" if c.is_single_zone else f"✗ Multi ({len(c.zones)})"
    print(f"  #{i}: {c.strategy.value}")
    print(f"      {c.total_pallets}p | {c.adjusted_priority:.0f} adj. priority | {zone_label}")

Candidates Ranked by Combined Score:
  Weights: Priority=0.5, Utilization=0.3, Zone Coherence=0.2

  #1: dp_optimal
      8.71p | 1900071 adj. priority | ✗ Multi (2)
  #2: greedy_efficiency
      8.0p | 1899998 adj. priority | ✗ Multi (2)
  #3: greedy_zone_caba
      8.78p | 1000126 adj. priority | ✓ Single
  #4: greedy_zone_west
      7.94p | 1000099 adj. priority | ✓ Single
  #5: greedy_zone_north
      8.77p | 209 adj. priority | ✓ Single
  #6: greedy_zone_south
      8.18p | 175 adj. priority | ✓ Single


In [28]:
# Top 5 candidates with full breakdown
top_5 = get_top_n_candidates(ranked_candidates, 5)

print("\n" + "=" * 70)
print("TOP 5 DISPATCH CANDIDATES")
print("=" * 70)

for i, c in enumerate(top_5, 1):
    print(f"\n{'─' * 70}")
    print(f"Rank #{i}: {c.candidate_id}")
    print(f"{'─' * 70}")
    print(f"Strategy:      {c.strategy.value}")
    print(f"Orders:        {len(c.order_ids)}")
    print(f"Pallets:       {c.total_pallets} ({c.utilization_pct}% utilization)")
    print(f"Priority:      {c.total_priority:.2f} (raw)")
    print(f"Adjusted:      {c.adjusted_priority:.2f} (after {c.zone_dispersion_penalty}x zone penalty)")
    print(f"Zones:         {', '.join(c.zones)}")
    print(f"Zone Breakdown: {c.zone_breakdown}")
    print(f"Mandatory:     {c.mandatory_count} orders included")
    print(f"\nOrders in this dispatch:")
    for order in c.orders:
        mand = "★" if order["is_mandatory"] else " "
        print(f"  {mand} {order['order_id']}: {order['client_name'][:25]:<25} | "
              f"{order['pallets']:.2f}p | {order['priority_score']:.1f}pri | {order['zone_id']}")


TOP 5 DISPATCH CANDIDATES

──────────────────────────────────────────────────────────────────────
Rank #1: DISP-20260119-DP_OPT-1710
──────────────────────────────────────────────────────────────────────
Strategy:      dp_optimal
Orders:        3
Pallets:       8.71 (108.9% utilization)
Priority:      2000074.46 (raw)
Adjusted:      1900070.74 (after 0.95x zone penalty)
Zones:         WEST_ZONE, CABA
Zone Breakdown: {'WEST_ZONE': 2, 'CABA': 1}
Mandatory:     2 orders included

Orders in this dispatch:
  ★ ORD-AF4525B0: Distribuidora Pampa       | 4.06p | 999999.0pri | WEST_ZONE
  ★ ORD-930E42E5: Supermercado Don Pedro    | 3.94p | 999999.0pri | CABA
    ORD-63A5B648: Distribuidora del Sur     | 0.71p | 76.5pri | WEST_ZONE

──────────────────────────────────────────────────────────────────────
Rank #2: DISP-20260119-GREEDY-D1C6
──────────────────────────────────────────────────────────────────────
Strategy:      greedy_efficiency
Orders:        2
Pallets:       8.0 (100.0% utilization)

### Non-Mandatory Candidates Comparison

The following shows dispatch candidates generated **without mandatory orders**, allowing us to see the true priority scores without the 999999 boost from mandatory flags. This is useful for understanding which orders have the highest actual priority.

In [29]:
# Non-mandatory candidates summary (already generated earlier for charts)
print(f"Non-mandatory candidates: {len(unique_non_mandatory)} unique")
print("\nNon-Mandatory Candidates Summary:")
display(non_mandatory_summary_df)

Non-mandatory candidates: 7 unique

Non-Mandatory Candidates Summary:


,candidate_id,strategy,total_pallets,adjusted_priority,total_priority,utilization_pct,zones,zone_count,is_single_zone,order_count,mandatory_count,order_ids
6,DISP-20260119-DP_OPT-ABA4,dp_optimal,9.36,410.89,483.40,117.0,"NORTH_ZONE, WEST_ZONE, CABA",3,False,6,0,"ORD-FE3E2E80, ORD-B24A7359, ORD-5A5B6366, ORD-..."
0,DISP-20260119-GREEDY-DD1F,greedy_efficiency,8.40,387.05,455.35,105.0,"NORTH_ZONE, WEST_ZONE, CABA",3,False,6,0,"ORD-63A5B648, ORD-5FF28C36, ORD-FE3E2E80, ORD-..."
4,DISP-20260119-GREEDY-3538,greedy_zone_west,8.85,350.48,350.48,110.6,WEST_ZONE,1,True,5,0,"ORD-6E478AEA, ORD-5FF28C36, ORD-63A5B648, ORD-..."
5,DISP-20260119-GREEDY-E7E8,greedy_zone_spillover,6.79,338.99,338.99,84.9,WEST_ZONE,1,True,4,0,"ORD-6E478AEA, ORD-5FF28C36, ORD-63A5B648, ORD-..."
2,DISP-20260119-GREEDY-462E,greedy_zone_north,8.77,208.75,208.75,109.6,NORTH_ZONE,1,True,3,0,"ORD-AEEA364E, ORD-FE3E2E80, ORD-8061B3FC"
3,DISP-20260119-GREEDY-4F57,greedy_zone_south,8.18,175.02,175.02,102.2,SOUTH_ZONE,1,True,2,0,"ORD-A5A4DFFE, ORD-A85586B6"
1,DISP-20260119-GREEDY-7D01,greedy_priority,7.87,103.38,103.38,98.4,CABA,1,True,1,0,ORD-89270B2E


In [30]:
# Rank and show Top 5 Non-Mandatory Candidates
ranked_non_mandatory = rank_candidates(unique_non_mandatory, config)
top_5_non_mandatory = get_top_n_candidates(ranked_non_mandatory, 5)

print("\n" + "=" * 70)
print("TOP 5 DISPATCH CANDIDATES (WITHOUT MANDATORY)")
print("=" * 70)
print("These candidates show real priority scores without the 999999 mandatory boost")

for i, c in enumerate(top_5_non_mandatory, 1):
    print(f"\n{'─' * 70}")
    print(f"Rank #{i}: {c.candidate_id}")
    print(f"{'─' * 70}")
    print(f"Strategy:      {c.strategy.value}")
    print(f"Orders:        {len(c.order_ids)}")
    print(f"Pallets:       {c.total_pallets} ({c.utilization_pct}% utilization)")
    print(f"Priority:      {c.total_priority:.2f} (real priority, no mandatory boost)")
    print(f"Adjusted:      {c.adjusted_priority:.2f} (after {c.zone_dispersion_penalty}x zone penalty)")
    print(f"Zones:         {', '.join(c.zones)}")
    print(f"Zone Breakdown: {c.zone_breakdown}")
    print(f"\nOrders in this dispatch:")
    for order in c.orders:
        print(f"    {order['order_id']}: {order['client_name'][:25]:<25} | "
              f"{order['pallets']:.2f}p | {order['priority_score']:.1f}pri | {order['zone_id']}")


TOP 5 DISPATCH CANDIDATES (WITHOUT MANDATORY)
These candidates show real priority scores without the 999999 mandatory boost

──────────────────────────────────────────────────────────────────────
Rank #1: DISP-20260119-GREEDY-3538
──────────────────────────────────────────────────────────────────────
Strategy:      greedy_zone_west
Orders:        5
Pallets:       8.85 (110.6% utilization)
Priority:      350.48 (real priority, no mandatory boost)
Adjusted:      350.48 (after 1.0x zone penalty)
Zones:         WEST_ZONE
Zone Breakdown: {'WEST_ZONE': 5}

Orders in this dispatch:
    ORD-6E478AEA: Autoservicio El Trebol    | 3.88p | 99.8pri | WEST_ZONE
    ORD-5FF28C36: Autoservicio La Plaza     | 1.12p | 96.5pri | WEST_ZONE
    ORD-63A5B648: Distribuidora del Sur     | 0.71p | 76.5pri | WEST_ZONE
    ORD-5A5B6366: Autoservicio La Plaza     | 1.08p | 66.2pri | WEST_ZONE
    ORD-A9E437CA: Comercio Don Carlos       | 2.06p | 11.5pri | WEST_ZONE

──────────────────────────────────────────────

## 10. Export Candidates

In [31]:
# Create output directory
DISPATCH_DIR.mkdir(parents=True, exist_ok=True)

# Export all unique candidates
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = DISPATCH_DIR / f"dispatch_candidates_{timestamp}.json"

export_candidates_to_json(ranked_candidates, output_file)

print(f"Exported {len(ranked_candidates)} candidates to:")
print(f"  {output_file}")

Exported 6 candidates to:
  c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\output\dispatches\dispatch_candidates_20260119_003304.json


In [32]:
# Export top candidate separately
if top_5:
    top_file = DISPATCH_DIR / f"top_dispatch_{timestamp}.json"
    export_candidates_to_json([top_5[0]], top_file)
    print(f"\nTop candidate exported to:")
    print(f"  {top_file}")
    
# Show sample of exported JSON
print("\n📄 Sample Export Format:")
with open(output_file, "r") as f:
    data = json.load(f)
    # Show just first candidate summary
    if data["candidates"]:
        sample = data["candidates"][0]
        print(json.dumps({
            "candidate_id": sample["candidate_id"],
            "strategy": sample["strategy"],
            "summary": sample["summary"],
        }, indent=2))


Top candidate exported to:
  c:\Users\Santi\Desktop\CV\portafolio\Eco-Bags-Delivery-Optimizer\output\dispatches\top_dispatch_20260119_003304.json

📄 Sample Export Format:
{
  "candidate_id": "DISP-20260119-DP_OPT-1710",
  "strategy": "dp_optimal",
  "summary": {
    "total_pallets": 8.71,
    "total_priority": 2000074.46,
    "utilization_pct": 108.9,
    "order_count": 3,
    "zones": [
      "WEST_ZONE",
      "CABA"
    ],
    "zone_breakdown": {
      "WEST_ZONE": 2,
      "CABA": 1
    },
    "is_single_zone": false,
    "zone_dispersion_penalty": 0.95,
    "adjusted_priority": 1900070.74,
    "mandatory_included": true,
    "mandatory_count": 2
  }
}


In [34]:
# Save candidates to database for next phases
# First, reload the database module to get new models and methods
import src.database
importlib.reload(src.database)
from src.database import get_database_manager

# Recreate db connection with new methods
db = get_database_manager(DB_PATH)

# Recreate tables to include new dispatch_candidates tables
db.create_tables()

# Generate batch ID for this run
batch_id = f"BATCH-{timestamp}"

# Save ranked candidates to database
saved_count = db.save_dispatch_candidates(ranked_candidates, batch_id, ranked=True)
print(f"\n💾 Saved {saved_count} candidates to database")
print(f"   Batch ID: {batch_id}")

# Also save non-mandatory candidates for comparison
batch_id_nm = f"BATCH-NM-{timestamp}"
saved_count_nm = db.save_dispatch_candidates(ranked_non_mandatory, batch_id_nm, ranked=True)
print(f"\n💾 Saved {saved_count_nm} non-mandatory candidates to database")
print(f"   Batch ID: {batch_id_nm}")

# Verify data was saved
print("\n📊 Verification - Latest candidates in database:")
latest = db.get_latest_dispatch_candidates()
for c in latest[:3]:
    print(f"   #{c.rank}: {c.strategy} - {c.total_pallets}p, {c.adjusted_priority:.0f} adj.pri")


💾 Saved 6 candidates to database
   Batch ID: BATCH-20260119_003304

💾 Saved 7 non-mandatory candidates to database
   Batch ID: BATCH-NM-20260119_003304

📊 Verification - Latest candidates in database:
   #1: greedy_zone_west - 8.85p, 350 adj.pri
   #2: dp_optimal - 9.36p, 411 adj.pri
   #3: greedy_efficiency - 8.4p, 387 adj.pri


## Summary

This notebook demonstrated:

1. **Order Analysis**: Loaded pending orders with priority scores and zone assignments
2. **Mandatory Handling**: Identified must-include orders and handled overflow by selecting subset
3. **Zone Distribution**: Analyzed order distribution across zones
4. **Strategy Demo**: Walked through zone spillover strategy step-by-step
5. **Candidate Generation**: Generated candidates using 11 different strategies
6. **Deduplication**: Removed duplicate solutions from different strategies
7. **Comparison**: Visualized candidates by priority, utilization, and zones (with and without mandatory)
8. **Zone Coherence**: Compared single-zone vs multi-zone solutions
9. **Ranking**: Applied weighted ranking to select top candidates
10. **Export**: Saved candidates in JSON format and to database

### Database Output

The following tables were populated for use in next phases:

| Table | Description |
|-------|-------------|
| `dispatch_candidates` | Generated dispatch candidates with strategy, pallets, priority, zones |
| `dispatch_candidate_orders` | Relationship between candidates and orders |

### Next Steps

The exported dispatch candidates are ready for:
- **Phase 5**: Route optimization using TSP/VRP to determine optimal delivery sequence
- **Phase 6**: Time window validation and schedule generation